# IBM Applied Data Science Capstone

## WEEK 4: PEER-GRADED ASSIGNMENT

__Author:__ Bart Onkenhout  
__Created:__ 2019-04-02  
__Modified:__ 2019-04-02  

### Purpose
This Python notebook will primarily be used for the capstone project in Weeks 1 through 5 of the IBM Applied Data Science Capstone, which counts towards the IBM Data Science Professional Certificate.

### Assignment
This week's assignment is to create a notebook in which we:  
1) clearly define a problem or idea of our choice that would require leveraging the Foursquare location API to solve or execute;  
2) describe our target audience and why they would care about our problem;  
3) summarize this in an Introduction/Business Problem section as __Part I__ of a final report;  
4) push the resulting notebook to GitHub.

---
# SECTION I: INTRODUCTION

### Abstract
In the previous capstone assignment in Week 3, I spent a signinificant amount of time exploring the city of __Toronto__, enriching the neighborhoods with venue data from the <a href='http://developer.foursquare.com/'>Foursquare API</a>. A nice isolated exercise to flex some of the Python muscles that have lain long-dormant. However, Toronto is really more of a proof-of-concept -- barring some out-of-the-blue job offer in Canada I don't think I'll be moving up to the Great White North. Firstly, I'm not really a cold-weather person; and secondly, I'm not sure I have enough points to qualify for an <a href='https://www.canada.ca/en/immigration-refugees-citizenship/services/immigrate-canada/express-entry.html'>ExpressEntry visa</a> ;). Canada -- you're lovely, but you're also buried in snow 80% of the time, so no thanks!

<img src='https://upload.wikimedia.org/wikipedia/commons/d/d9/Flag_of_Canada_%28Pantone%29.svg' height='192' width='96' />



So perhaps it's time to kick our analysis up another notch. In the near future, I plan to move to Europe for a work assignment. However, my employer has headquarters in several European cities. So which work assignment should I apply for? The one in <a href='https://en.wikipedia.org/wiki/Frankfurt'>Frankfurt</a>? <a href='https://en.wikipedia.org/wiki/Barcelona'>Barcelona</a>? Perhaps one in <a href='https://en.wikipedia.org/wiki/Amsterdam'>Amsterdam</a>? I'd like to find the most similar cities in Europe and do the following:  

1) find the cities in which my employer has headquarters,  
2) enrich this set of cities with data from the Foursquare API,  
3) see how this compares with my current home city of Chicago in terms of venue diversity,  
4) make a decision based on this data, and  
5) show _you_, my lovely audience (and classmates) my reasoning!  

### Why is this important?
So who really cares? Well, there's a couple of reasons why this is important.

First, it's always good to make __evidence-driven decisions__. With increasing globalization, it's important to spend one's time wisely and get the most out of every place we travel to. However, there's an increasing problem of __information overload__ -- known in psychology as the <a href='https://en.wikipedia.org/wiki/The_Paradox_of_Choice'>Paradox of Choice</a>. I'm a bit on the older side and no longer have the luxury of 'finding myself' while backpacking for months through the Continent by rail like a <a href='https://en.wikipedia.org/wiki/Bohemianism'>bohemian</a>. No--I've got _bills!_ _Student loans!_ A bedtime of _9:30 PM!_ This means I'm looking for maximum payoff with my time. And third, I __really, really__ like food, so I want to find a place that's like Chicago in its diversity of cuisine offerings. (For my fellow students who are not familiar with the US, Chicago has a reputation as a mecca for food!)

So, because I definitely can't _eat_ my way through Europe I'll need to be able to filter out some of the places I definitely _don't_ want to go. This will help me to continuously refine my selection until I have a shortlist of, say, three cities that I can choose to focus on before settling on a final decision. And data science will help me to arrive at this shortlist in a systematic, reproducible way -- unlocking the secrets of the Foursquare API in a way that we never could with Excel!.

__So let's get to it!__

---
# SECTION II: DATA

### In a nutshell
In a nutshell, the data I'll need is fairly straightforward in concept. We want:

- __Foursquare API data__ for looking up venues in each city  
- a __list of European cities__ in which my employer has offices as a base comparison set
- __geolocation data__ of each city so we can plot everything on a map
- __GeoJSON shapes__ of each city so we can use a GeoJson layer to outline each city on the map


### 1. Foursquare API data
This one is fairly straightforward. I've already signed up for an API key at <a href='http://developer.foursquare.com'>http://developer.foursquare.com</a>, which will allow me to make a number of calls (at no charge) to get the information I need from the various <a href='https://developer.foursquare.com/docs/api/endpoints'>endpoints</a> that are exposed by the API. The most interesting information I'll want is the __venue data__ in a certain radius around each city.

We will use the `explore` endpoint to get venue recommendations based on a latitude and longitude passed in the `ll` parameter. However, according to the documentation, the non-premium API only returns __50 venues__ at a time. I'll have to use the `offset` parameter to page through the different results.

As an example, here is a list of 10 venues returned near the Willis Tower (formerly Sears Tower) in Chicago, IL (41.8789° N, 87.6359° W):

`https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={API_VERSION}&ll=41.8789,-87.6359&radius={radius}`

Where `CLIENT_ID` and `CLIENT_SECRET` are the API ID and API key, `API_VERSION` is the version date of the API, `ll` is the latitude and longitude of the location I want to search around in a radius of `radius` meters.


### 2. List of European cities
This list is obviously only available to employees of my company. Luckily, we have a central company directory that has the following data for each office:

- `LOCATION`: Office's official location name in our directory
- `CITY`    : City of the office
- `COUNTRY` : Country of the office

Fairly straightforward. I'll be loading each of these as a CSV file into a dict keyed on `LOCATION`, with fields like in the following example:

`offices['location']: {'city':'Frankfurt', 'country':'Germany'}`

### 3. Geolocation data
This one is a bit tougher, since Google made their Google Maps API a <a href='https://developers.google.com/maps/documentation/geocoding/usage-and-billing'>premium product</a>. However, according to Google there are USD 200 in free credits allowed every month, and the prices start at only USD 5 per 1000 requests. So if I sign up for a free account, I can make up to 40,000 requests before I need to start paying. This is one possible route.

Another possible route would be to use the OpenStreetMaps alternative API, <a href='https://wiki.openstreetmap.org/wiki/Nominatim'>Nominatim</a>. I do recall from previous usage that OSM can be finicky in which inputs it does and does not accept, so geocoding our European cities may prove difficult. Depending on the number of cities available, I may want to do this offline in Excel by hand-querying the Google Maps website, rather than using the API.

In any case, the Nominatim API can be queried using the following syntax:

`https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat=-34.4391708&lon=-58.7064573`

This will return a JSON object containing geographic information on the location(s) at the specified latitude and longitude (in this example, the Aramburu highway in Argentina). The JSON object looks like this:

```
{
  "place_id":"134140761",
  "licence":"Data © OpenStreetMap contributors, ODbL 1.0. http:\/\/www.openstreetmap.org\/copyright",
  "osm_type":"way",
  "osm_id":"280940520",
"lat":"-34.4391708",
  "lon":"-58.7064573",
  "place_rank":"26",
  "category":"highway",
  "type":"motorway",
  "importance":"0.1",
  "addresstype":"road",
  "display_name":"Autopista Pedro Eugenio Aramburu, El Triángulo, Partido de Malvinas Argentinas, Buenos Aires, 1.619, Argentina",
  "name":"Autopista Pedro Eugenio Aramburu",
  "address":{
    "road":"Autopista Pedro Eugenio Aramburu",
    "village":"El Triángulo",
    "state_district":"Partido de Malvinas Argentinas",
    "state":"Buenos Aires",
    "postcode":"1.619",
    "country":"Argentina",
    "country_code":"ar"
  },
  "boundingbox":["-34.44159","-34.4370994","-58.7086067","-58.7044712"]
}
```

### 4. GeoJSON shapes
Luckily, the European Commission has an excellent <a href='https://ec.europa.eu/eurostat/web/gisco/geodata'>website</a> that may help us with this. In its <a href='https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/urban-audit#ua11-14'>Urban Audit</a> section, we can find `.shp` boundary files for each city that falls under the EC's watchful eye. I'll use <a href='https://medium.com/dataexplorations/generating-geojson-file-for-toronto-fsas-9b478a059f04'>this article</a> to convert the `.shp` files into GeoJSON files, as I did for Part 3 of the Week 3 assignment, using the open-source <a href='https://qgis.org/en/site/'>QGIS</a> software.